라이브러리

In [31]:
import pandas as pd
import numpy as np
from keras.utils import *
from sklearn.preprocessing import *
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
from keras.models import load_model
from keras.layers import LSTM, Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.dates import DateFormatter


# 한글 글꼴 경로 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 원하는 한글 글꼴 파일 경로로 수정해주세요
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

데이터셋 설정

In [32]:
#자포니카 훈련 데이터
japonica_training_food_supply_tb = pd.read_csv("E:/kimwoojae/Research/DO Regressor/dataset/japonica_training_food_supply_tb.csv")
japonica_training_sensor_val_tb = pd.read_csv("E:/kimwoojae/Research/DO Regressor/dataset/japonica_training_sensor_val_tb.csv")

#자포니카 검증 데이터
japonica_validation_food_supply_tb = pd.read_csv("E:/kimwoojae/Research/DO Regressor/dataset/japonica_validation_food_supply_tb.csv")
japonica_validation_sensor_val_tb = pd.read_csv("E:/kimwoojae/Research/DO Regressor/dataset/japonica_validation_sensor_val_tb.csv")

In [33]:
#자포니카 훈련 데이터 시계열 변환
japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].apply(str)
japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
japonica_training_food_supply_tb['feed_dt'] = pd.to_datetime(japonica_training_food_supply_tb['feed_dt'], format='%Y%m%d%H%M', errors='raise')
japonica_training_food_supply_tb["feed_dt"] = japonica_training_food_supply_tb["feed_dt"].apply(str)
japonica_training_food_supply_tb["feed_dt"] = japonica_training_food_supply_tb["feed_dt"].str.slice(start=0, stop=16)

#자포니카 검증 데이터 시계열 변환
japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].apply(str)
japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
japonica_validation_food_supply_tb['feed_dt'] = pd.to_datetime(japonica_validation_food_supply_tb['feed_dt'], format='%Y%m%d%H%M', errors='raise')
japonica_validation_food_supply_tb["feed_dt"] = japonica_validation_food_supply_tb["feed_dt"].apply(str)
japonica_validation_food_supply_tb["feed_dt"] = japonica_validation_food_supply_tb["feed_dt"].str.slice(start=0, stop=16)

In [34]:
# 자포니카 훈련 데이터 및 시계열 데이터 병합
japonica_training_tankAll = pd.merge(left = japonica_training_sensor_val_tb, right = japonica_training_food_supply_tb, how = "left", left_on = ["farm_id","tank_id", "mea_dt"], right_on = ["farm_id","tank_id", "feed_dt"])
japonica_validation_tankAll = pd.merge(left = japonica_validation_sensor_val_tb, right = japonica_validation_food_supply_tb, how = "left", left_on = ["farm_id","tank_id", "mea_dt"], right_on = ["farm_id","tank_id", "feed_dt"])

In [35]:
#자포니카 훈련 및 검증 데이터 시계열 변환
japonica_training_tankAll['mea_dt'] = pd.to_datetime(japonica_training_tankAll['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')
japonica_validation_tankAll['mea_dt'] = pd.to_datetime(japonica_validation_tankAll['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')

In [36]:
# 날짜 데이터를 인덱스로 전환
japonica_training_tankAll.set_index('mea_dt', inplace=True)
japonica_validation_tankAll.set_index('mea_dt', inplace=True)

In [37]:
# 인덱스 순으로 데이터를 정렬
japonica_training_tankAll = japonica_training_tankAll.sort_index()
japonica_validation_tankAll = japonica_validation_tankAll.sort_index()

In [8]:
# 자포니카 훈련 및 검증 데이터의 각 탱크 번호
tank = 1
japonica_training_tank = japonica_training_tankAll[japonica_training_tankAll['tank_id']==tank]
japonica_validation_tank = japonica_validation_tankAll[japonica_validation_tankAll['tank_id']==tank]

In [9]:
# 사용할 features 선택
japonica_training_tank_features = japonica_training_tank[['do_mg','do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']]
japonica_validation_tank_features = japonica_validation_tank[['do_mg','do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']]

# nan 값 처리 (먹이를 주지 않았을 경우는 급여량이 0이니까)
japonica_training_tank_features = japonica_training_tank_features.fillna(0)
japonica_validation_tank_features = japonica_validation_tank_features.fillna(0)

In [10]:
# 오토인코딩을 위해 데이터를 reshape.
japonica_training_tank_features = np.asarray(japonica_training_tank_features, dtype=np.float64)
japonica_training_tank_features = japonica_training_tank_features.reshape((-1, 1, 9))

In [14]:
from keras.layers import Input, SimpleRNN, TimeDistributed, RepeatVector
from keras.models import Model
from keras.callbacks import EarlyStopping


input_layer = Input(shape=(1, 9))
encoded = SimpleRNN(units=9, activation='relu')(input_layer)
decoded = RepeatVector(1)(encoded)
decoded = SimpleRNN(units=9, activation='relu', return_sequences=True)(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

# 오토인코더 학습
autoencoder.fit(japonica_training_tank_features, japonica_training_tank_features, epochs=100, batch_size=32, callbacks=[early_stopping])

# 학습된 오토인코더 모델을 사용하여 노이즈 제거
denoised_seq = autoencoder.predict(japonica_training_tank_features)

Epoch 1/100
6961/6961 [==============================] - 12s 1ms/step - loss: 460.5569
Epoch 2/100
6961/6961 [==============================] - 10s 1ms/step - loss: 9.5681
Epoch 3/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9274
Epoch 4/100
6961/6961 [==============================] - 11s 2ms/step - loss: 8.9252
Epoch 5/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9241
Epoch 6/100
6961/6961 [==============================] - 11s 2ms/step - loss: 8.9230
Epoch 7/100
6961/6961 [==============================] - 11s 2ms/step - loss: 8.9221
Epoch 8/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9213
Epoch 9/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9207
Epoch 10/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9201
Epoch 11/100
6961/6961 [==============================] - 10s 1ms/step - loss: 8.9195
Epoch 12/100
6961/6961 [==============================] - 10s

In [46]:
import numpy as np

# denoised_seq 배열의 크기 확인
n, m, o = denoised_seq.shape

# 3차원 데이터를 2차원으로 변경
denoised_seq2 = denoised_seq.reshape(n, m*o)
print(denoised_seq2)

[[ 0.        30.331379   5.527341  ...  0.         0.         0.       ]
 [ 0.        30.332975   5.5264225 ...  0.         0.         0.       ]
 [ 0.        30.430082   5.509416  ...  0.         0.         0.       ]
 ...
 [ 0.        28.703403   5.156518  ...  0.         0.         0.       ]
 [ 0.        28.702442   5.1522584 ...  0.         0.         0.       ]
 [ 0.        28.702673   5.1540275 ...  0.         0.         0.       ]]


In [48]:
japonica_training_tank_features = pd.DataFrame(denoised_seq2, columns=japonica_validation_tank_features.columns, index=list(japonica_validation_tank_features.index.values))
japonica_training_tank_features

,do_mg,do_temp,ph,orp,co2_mg,air_oxy,light_ma,feed_quantity,water_quantity
2021-07-15 19:03:00,0.0,30.331379,5.527341,160.006561,14.599540,0.000000,0.0,0.0,0.0
2021-07-15 19:04:00,0.0,30.332975,5.526423,160.106598,14.599560,0.000000,0.0,0.0,0.0
2021-07-15 19:05:00,0.0,30.430082,5.509416,160.306564,14.499483,0.000000,0.0,0.0,0.0
2021-07-15 19:06:00,0.0,30.332527,5.524225,160.506516,14.499514,0.000000,0.0,0.0,0.0
2021-07-15 19:07:00,0.0,30.332541,5.524223,160.506577,14.499537,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2022-01-06 19:10:00,0.0,28.703569,5.157859,215.977814,32.495743,11.408190,0.0,0.0,0.0
2022-01-06 19:12:00,0.0,28.703447,5.158453,216.177856,32.495773,11.308457,0.0,0.0,0.0
2022-01-06 19:13:00,0.0,28.703403,5.156518,216.477783,32.495762,11.307858,0.0,0.0,0.0
2022-01-06 19:14:00,0.0,28.702442,5.152258,216.577576,32.495655,11.508538,0.0,0.0,0.0


In [ ]:
# 독립 변수, 종속 변수 분리
japonica_training_tank_features_X = japonica_training_tank_features[['do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']]
japonica_training_tank_features_y = japonica_training_tank_features[['do_mg']]
japonica_validation_tank_features_X = japonica_validation_tank_features[['do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']]
japonica_validation_tank_features_y = japonica_validation_tank_features[['do_mg']]

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(japonica_training_tank_features.index, japonica_training_tank_features[['do_mg']], label='actual')  # x축에 년월일, y축에 값
plt.xlabel('날짜')
plt.ylabel('DO')
plt.title('시계열')
plt.legend(['y_test','X_pred'])
plt.show()

In [ ]:
# LSTM학습을 위해 데이터 reshape를 해야함. reshape를 위해 배열형으로 변환
japonica_training_tank_features_X_reshape = np.asarray(japonica_training_tank_features_X, dtype=np.float64)
japonica_validation_tank_features_X_reshape = np.asarray(japonica_validation_tank_features_X, dtype=np.float64)

# 데이터를 3항으로 reshape. (batch_size, timesteps, features)
# batch_size: 한 번에 모델에 입력되는 샘플의 개수
# timesteps: 입력되는 시퀀스 데이터의 길이(시간축)
# features: 입력되는 데이터의 특성 개수
# 말이 어려우니까 쉽게 쓰면 (얼마 만큼의 샘플을, 시간 당 몇 개씩, 항목이 몇 개인가)
# (-1 : 있는 만큼의 샘플을, 1 : 시간당 1개씩, 8 : 피처는 8개입니다.)
japonica_training_tank_features_X_reshape = japonica_training_tank_features_X_reshape.reshape((-1, 1, 8))
japonica_validation_tank_features_X_reshape = japonica_validation_tank_features_X_reshape.reshape((-1, 1, 8))

In [ ]:
# shape확인
nCar = japonica_training_tank_features_X_reshape.shape[0] # 데이터 개수
nVar = japonica_training_tank_features_X_reshape.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar = japonica_training_tank_features_y.shape[0] # 데이터 개수
nVar = japonica_training_tank_features_y.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

# shape확인
nCar = japonica_validation_tank_features_X_reshape.shape[0] # 데이터 개수
nVar = japonica_validation_tank_features_X_reshape.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar = japonica_validation_tank_features_y.shape[0] # 데이터 개수
nVar = japonica_validation_tank_features_y.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

In [ ]:
# r_squared 평가 함수
def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - ss_res/(ss_tot + K.epsilon()))

LSTM 모델 학습 (학습시에만 사용, 그외에는 불필요)

In [ ]:
# LSTM 모델 생성
model = Sequential()
# 결과값이 128개 -> 64개 -> 32개 -> 1개(회귀)
# input_shape=(timesteps, input_dim)
# timesteps : 시계열 데이터의 시간 스텝 수.
#       예를 들어, 1분 단위로 측정한 센서 데이터가 있다면 timesteps는 60.
# input_dim : 특성(feature)의 수.
model.add(LSTM(128, input_shape=(1, 8), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='linear'))
 
model.summary()
 
# mae와 r_squared 평가함수 추가.
model.compile(optimizer='adam', loss='mse', metrics=['mae', r_squared])

In [ ]:
model_name = 'test230405-01_lstmJaponica7.h5'
hist = model.fit(japonica_training_tank_features_X_reshape, japonica_training_tank_features_y, epochs = 200, batch_size = 64, validation_data=(japonica_validation_tank_features_X_reshape, japonica_validation_tank_features_y))
model.save(model_name)

모델 평가

In [ ]:
# 모델 평가 함수
def grape(X_test, y_test, X_pred):
    # 그래프 생성
    plt.figure(figsize=(15, 7))
    plt.plot(X_test.index, y_test, label='actual')  # x축에 년월일, y축에 값
    plt.plot(X_test.index, X_pred, color='red', label='prediction')  # x축에 년월일, y축에 시분
    plt.xlabel('날짜')
    plt.ylabel('DO')
    plt.title('시계열')
    plt.legend(['y_test','X_pred'])
    plt.show()
    
def eva(X_test_shape, y_test):
    # 평가 생성
    result = model.evaluate(X_test_shape, y_test)
    print("MSE // MAE // R-squared ", result)

In [ ]:
# r_squared가 따로 만들어서 추가해준 함수기 때문에 불러올 때도 추가해서 불러야한다.
model = load_model(model_name, custom_objects={'r_squared': r_squared})

In [ ]:
# 가중치를 통해 변수 중요도 획득
weights = model.layers[0].get_weights()[0]
print(weights)


# 변수 중요도 시각화
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(range(8), weights[:, 0])
ax.set_xticks(range(8))
ax.set_xticklabels(['do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity'])
ax.set_ylabel('Weight')
ax.set_title('Variable Importance')
plt.show()

In [ ]:
# japonica_validation_tank_features_X_reshape 검증모델 예측
japonica_validation_tank_features_X_pred = model.predict(japonica_validation_tank_features_X_reshape)

In [ ]:
grape(japonica_validation_tank_features_X, japonica_validation_tank_features_y, japonica_validation_tank_features_X_pred)
eva(japonica_validation_tank_features_X_reshape, japonica_validation_tank_features_y)

In [ ]:
# 하루 동안의 데이터 추출
one_day_data = japonica_validation_tank_features.loc['2021-08-27 00:00:00':'2021-08-27 23:59:59']
oneday_X_test = one_day_data[['do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']]
oneday_y_test = one_day_data[['do_mg']]

oneday_X_test_reshape = np.asarray(oneday_X_test, dtype=np.float64)
oneday_X_test_reshape = oneday_X_test_reshape.reshape((-1, 1, 8))

# shape확인
nCar = oneday_X_test_reshape.shape[0] # 데이터 개수
nVar = oneday_X_test_reshape.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar = oneday_y_test.shape[0] # 데이터 개수
nVar = oneday_y_test.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

oneday_pred = model.predict(oneday_X_test_reshape)

In [ ]:
grape(oneday_X_test, oneday_y_test, oneday_pred)

In [ ]:
japonica_training_tank['year_month_day'] = japonica_training_tank.index.strftime('%Y-%m-%d')  # 년월일만 추출
japonica_training_tank['hour_minute'] = japonica_training_tank.index.strftime('%H:%M')  # 시분만 추출
japonica_cross_tab_tank = pd.crosstab(index=japonica_training_tank['hour_minute'], columns=japonica_training_tank['year_month_day'], values=japonica_training_tank['do_mg'], aggfunc='sum')

japonica_cross_tab_tank